In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Biblographic Coupling and Co-Citation Code

1. Load Mult.Py

In [0]:
!pip install snap
!pip install gc
!pip install pydot


In [0]:
import snap

COCI = snap.LoadEdgeList(snap.PNEANet, "/content/drive/My Drive/data/Cit-HepTh.txt", 0, 1)

COCI.AddStrAttrN("Layer")
COCI.AddIntAttrN("Visited")
COCI.AddStrAttrE("InterOrIntra")
COCI.AddStrAttrE("Dimension")
COCI.AddIntAttrE("Weight")

for N in COCI.Nodes():
    COCI.AddStrAttrDatN(N, "Article", "Layer")
    COCI.AddIntAttrDatN(N, 0, "Visited")


FOut = snap.TFOut("/content/drive/My Drive/data/DOIMulti.graph")
COCI.Save(FOut)
FOut.Flush()

print "Citation Graph saved"

#no of nodes
print "Nodes: %d" % COCI.GetNodes()

#no of edges
print "Edges: %d" % COCI.GetEdges()

2. loadMultiEdgeLabel.py

In [0]:
import snap
import gc

FIn = snap.TFIn("/content/drive/My Drive/data/DOIMulti.graph")
G4 = snap.TNEANet.Load(FIn)
print "Nodes: %d" % G4.GetNodes()
print "Edges: %d" % G4.GetEdges()


for EI in G4.Edges():
    G4.AddStrAttrDatE(EI, "Intra", "InterOrIntra")
    G4.AddStrAttrDatE(EI, "Citation", "Dimension")
    G4.AddIntAttrDatE(EI, 1, "Weight")
    gc.collect()

FOut = snap.TFOut("/content/drive/My Drive/data/DOIMultiEdgeLabels.graph")
G4.Save(FOut)
FOut.Flush()

3.makeBib.py

In [0]:
import snap
import gc
import pydot



P = pydot.Dot()
i = 0
DEBUG = 0
FIn = snap.TFIn("/content/drive/My Drive/data/DOIMultiEdgeLabels.graph")
M = snap.TNEANet.Load(FIn)
print ("Nodes: %d" % M.GetNodes())
print ("Edges: %d" % M.GetEdges())
"""

for EI in M.Edges():
    if M.GetStrAttrDatE(EI,"Dimension") == "Citation":
        P.add_edge(pydot.Edge(EI.GetSrcNId(),EI.GetDstNId()))
    else:
        P.add_edge(pydot.Edge(EI.GetSrcNId(), EI.GetDstNId(), label="Bibliographic Coupling", labelfontcolor="#009933", fontsize="10.0", color="blue", arrowhead="none"))

"""
for src in M.Nodes():
    if((M.GetIntAttrDatN(src, "Visited") == 0) ):
        i = i + 1
        gc.collect()
        outDeg = src.GetOutDeg()
        outEIdList = []
        #print lst
        for i in range(outDeg):
            gc.collect()
            outEIdList.append(src.GetOutEId(i))
        for citedEdge in outEIdList:
            gc.collect()
            if M.GetStrAttrDatE(M.GetEI(citedEdge),"Dimension") == "Citation":
                cited = M.GetNI(M.GetEI(citedEdge).GetDstNId())#Src
                inDeg = cited.GetInDeg()
                inEIdList = []
                #print lst
                for i in range(inDeg):
                    gc.collect()
                    inEIdList.append(cited.GetInEId(i))
                for citedOutE in inEIdList:
                    gc.collect()
                    dst = M.GetEI(citedOutE).GetSrcNId()#Dst
                    if M.GetStrAttrDatE(citedOutE,"Dimension") == "Citation":

                        if src.GetId() < dst:
                            if M.IsEdge(src.GetId(), dst) and M.GetStrAttrDatE(M.GetEI(src.GetId(), dst),"Dimension") == "Bibliographic Coupling":
                                M.AddIntAttrDatE(M.GetEI(src.GetId(), dst),M.GetIntAttrDatE(M.GetEI(src.GetId(), dst),"Weight")+1,"Weight")
                                M.AddIntAttrDatE(M.GetEI(dst, src.GetId()),M.GetIntAttrDatE(M.GetEI(dst, src.GetId()),"Weight")+1,"Weight")
                                if (DEBUG):
                                    P.add_edge(pydot.Edge(src.GetId(), dst, label="Bibliographic Coupling ( "+str(cited.GetId())+" )", labelfontcolor="#009933", fontsize="10.0", color="blue", arrowhead="none"))
                                                            
                            elif (M.IsEdge(src.GetId(), dst) and M.GetStrAttrDatE(M.GetEI(src.GetId(), dst),"Dimension") == "Citation") or M.IsEdge(src.GetId(), dst) == False:
                                Eid1 = M.AddEdge(src.GetId(), dst)
                                Eid2 = M.AddEdge(dst, src.GetId())
                                if (DEBUG):
                                    P.add_edge(pydot.Edge(src.GetId(), dst, label="Bibliographic Coupling ( "+str(cited.GetId())+" )", labelfontcolor="#009933", fontsize="10.0", color="blue", arrowhead="none"))
                                M.AddIntAttrDatE(Eid1,1,"Weight")
                                M.AddIntAttrDatE(Eid2,1,"Weight")                        
                                M.AddStrAttrDatE(Eid1,"Bibliographic Coupling","Dimension")
                                M.AddStrAttrDatE(Eid2,"Bibliographic Coupling","Dimension")
                                M.AddStrAttrDatE(Eid1, "Intra", "InterOrIntra")
                                M.AddStrAttrDatE(Eid2, "Intra", "InterOrIntra")

        M.AddIntAttrDatN(src,1,"Visited")
        FOut = snap.TFOut("/content/drive/My Drive/data/DOIMultiEdgeLabels.graph")
        M.Save(FOut)
        FOut.Flush()

#M.Dump()

"""
gc.collect()
for src in M.Nodes():
    inDeg = src.GetInDeg()
    inEIdList = []
    #print lst
    for i in range(inDeg):
        inEIdList.append(src.GetInEId(i))
        #print src.GetId(),inDeg, inEIdList, src.GetInEId(i)
    for citingEdge in inEIdList:
        if M.GetStrAttrDatE(M.GetEI(citingEdge),"Dimension") == "Citation":
            citing = M.GetNI(M.GetEI(citingEdge).GetSrcNId())
            outDeg = citing.GetOutDeg()
            outEIdList = []
            #print lst
            for i in range(outDeg):
                outEIdList.append(citing.GetOutEId(i))
                #print src.GetId(),outDeg, outEIdList, src.GetOutEId(i)
            for citedOutE in outEIdList:
                dst = M.GetEI(citedOutE).GetDstNId()
                if M.GetStrAttrDatE(citedOutE,"Dimension") == "Citation":
                    if src.GetId() < dst:
                        if M.IsEdge(src.GetId(), dst) and M.GetStrAttrDatE(M.GetEI(src.GetId(), dst),"Dimension") == "Co-Citation":
                            M.AddIntAttrDatE(M.GetEI(src.GetId(), dst),M.GetIntAttrDatE(M.GetEI(src.GetId(), dst),"Weight")+1,"Weight")
                            M.AddIntAttrDatE(M.GetEI(dst, src.GetId()),M.GetIntAttrDatE(M.GetEI(dst, src.GetId()),"Weight")+1,"Weight")
                            P.add_edge(pydot.Edge(src.GetId(), dst, label="Co-Citation ( "+str(citing.GetId())+" )", labelfontcolor="#009933", fontsize="10.0", color="red", arrowhead="none"))                        
                        elif (M.IsEdge(src.GetId(), dst) and M.GetStrAttrDatE(M.GetEI(src.GetId(), dst),"Dimension") == "Citation") or M.IsEdge(src.GetId(), dst) == False:
                            '''
                            if M.IsEdge(src.GetId(), dst): 
                                print M.IsEdge(src.GetId(), dst), M.GetStrAttrDatE(M.GetEI(src.GetId(), dst),"Dimension"), src.GetId() , dst, citing
                            else:
                                print M.IsEdge(src.GetId(), dst), src.GetId() , dst, citing
                            '''
                            Eid1 = M.AddEdge(src.GetId(), dst)
                            Eid2 = M.AddEdge(dst, src.GetId())
                            P.add_edge(pydot.Edge(src.GetId(), dst, label="Co-Citation ( "+str(citing.GetId())+" )", labelfontcolor="#009933", fontsize="10.0", color="red", arrowhead="none"))
                            M.AddIntAttrDatE(Eid1,1,"Weight")
                            M.AddIntAttrDatE(Eid2,1,"Weight")                        
                            M.AddStrAttrDatE(Eid1,"Co-Citation","Dimension")
                            M.AddStrAttrDatE(Eid2,"Co-Citation","Dimension")
                            M.AddStrAttrDatE(Eid1, "Intra", "InterOrIntra")
                            M.AddStrAttrDatE(Eid2, "Intra", "InterOrIntra")

gc.collect()

"""
#snap.SaveEdgeList(M, 'network_mygraph.txt')


"""
P.write('temp.dot')


im_name = "temp.png"
P.write_png(im_name)


NidName = snap.TIntStrH()
for i in M.Nodes():
	NidName[i.GetId()] = str(i.GetId())

snap.DrawGViz(M, snap.gvlDot, "G2.png", "G2", NidName)


CmtyV = snap.TCnComV()
modularity = snap.CommunityGirvanNewman(M, CmtyV)
#for Cmty in CmtyV:
    #print "Community: "
    #for NI in Cmty:
        #print NI
print "The modularity of the network is %f" % modularity

"""

## Louvain Community Detection Algorithm

● The Louvain algorithm is a greedy algorithm which gives different communities on multiple run on the same dataset.The louvain Algorithm is based on hierarchical clustering which merges multiple communities to single nodes based on modularity
● We have imported the louvain algorithm library in python using networkx.
● This is the Code for Louvain Algorithm

In [2]:
!pip install networkx
!pip install community

In [0]:
import networkx as nx
from community import community_louvain


G = nx.read_edgelist('/content/drive/My Drive/data/com-dblp.ungraph.txt', create_using = nx.Graph(), nodetype = int)

parts = community_louvain.best_partition(G, resolution = 5, randomize = 0)
values = [parts.get(node) for node in G.nodes()]

a = []
b = []

i=0
for item in parts:
    a.append(item)
    #a[i] = item
    i = i + 1

i=0
for item2 in values:
    b.append(item2)
    #b[i] = item2
    i = i + 1

with open('/content/drive/My Drive/data/LovaindblpCommunities.txt', 'a') as f:
#f= open("a.txt","w+")
    for i in range(len(a)):
        f.write(str(a[i]) + '\t ' + str(b[i]) + '\n')

● By running this code, we found the communities on DBLP Network
● The output shows the community has been assigned to each node. The first column in each row defines nodes and 2nd column its respective community.

## Leiden Community Detection Algorithm

● Leiden algorithm gives communities which are densely connected internally.
● We have implemented leiden algorithm using igraph
● We have downloaded Anaconda Navigator and installed the Leiden Algorithmlibrary which was done successfully.
● This is the Code for Leiden Algorithm

In [5]:
!pip install leidenalg
!pip install igraph
!pip install matplotlib

     |████████████████████████████████| 2.4MB 2.7MB/s 
     |████████████████████████████████| 3.2MB 21.3MB/s 
     |████████████████████████████████| 122kB 2.8MB/s 


In [7]:
import leidenalg
import igraph as ig
import matplotlib.pyplot as plt

G = Graph.Read_Edgelist('/content/drive/My Drive/data/com-dblp.ungraph.txt', directed=False)

part = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition)

print (part)

file3 = open("/content/drive/My Drive/data/LeidenDblpCommunities.txt", "a")

for items in part:
    for elements in items:
        #print (str(elements))
        file3.write(str(elements))
        file3.write("\t")
    file3.write("\n")
file3.close()

DeprecationWarning: ignored

● By running this code, we found the communities on DBLP Network
● The image attached below shows the text file form by above code each row represent community and numbers of nodes in that community.
● There are many communities that contain only 1 node.